In [1]:
import numpy as np
import pandas as pd
import zipfile
import json

In [25]:
path = "/Users/jeremynixon/Dropbox/python_new/NLP/aws/yelp_1000.txt"

In [26]:
def cut_characters(path):
    text = open(path).read().lower()
    char_indices = json.loads(open('/Users/jeremynixon/Dropbox/python_new/NLP/aws/char_indices.txt').read())
    indices_char = json.loads(open('/Users/jeremynixon/Dropbox/python_new/NLP/aws/indices_char.txt').read())
    not_in = []
    for i in list(set(text)):
        if i not in char_indices.keys():
            if i not in not_in:
                print i
            not_in.append(i)
    for char in not_in:
        text = text.translate(None, char)
    to_write = open(path, 'w')
    to_write.write(text)

In [27]:
cut_characters(path)

@
_
~


In [297]:
#correct this for where your data is saved (should be the same folder as every other file in this project)
path = "/Users/jeremynixon/Downloads/yelp_training_set.zip"

In [298]:
archive = zipfile.ZipFile(path, 'r')

In [210]:
archive.namelist()

['yelp_training_set/',
 'yelp_training_set/._yelp_training_set_business.json',
 'yelp_training_set/._yelp_training_set_checkin.json',
 'yelp_training_set/._yelp_training_set_review.json',
 'yelp_training_set/._yelp_training_set_user.json',
 'yelp_training_set/PaxHeader/',
 'yelp_training_set/PaxHeader/._yelp_training_set_business.json',
 'yelp_training_set/PaxHeader/._yelp_training_set_checkin.json',
 'yelp_training_set/PaxHeader/._yelp_training_set_review.json',
 'yelp_training_set/PaxHeader/._yelp_training_set_user.json',
 'yelp_training_set/PaxHeader/yelp_training_set_business.json',
 'yelp_training_set/PaxHeader/yelp_training_set_checkin.json',
 'yelp_training_set/PaxHeader/yelp_training_set_review.json',
 'yelp_training_set/PaxHeader/yelp_training_set_user.json',
 'yelp_training_set/yelp_training_set_business.json',
 'yelp_training_set/yelp_training_set_checkin.json',
 'yelp_training_set/yelp_training_set_review.json',
 'yelp_training_set/yelp_training_set_user.json']

In [211]:
archive.extractall("/Users/jeremynixon/Dropbox/python_new/NLP")

In [212]:
archive.close()

In [213]:
json_file = open('yelp_training_set/yelp_training_set_business.json')
json_str = json_file.read()
print json_str[:1000]

{"business_id": "rncjoVoEFUJGCUoC1JgnUA", "full_address": "8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345", "open": true, "categories": ["Accountants", "Professional Services", "Tax Services", "Financial Services"], "city": "Peoria", "review_count": 3, "name": "Peoria Income Tax Service", "neighborhoods": [], "longitude": -112.241596, "state": "AZ", "stars": 5.0, "latitude": 33.581867000000003, "type": "business"}
{"business_id": "0FNFSzCFP_rGUoJx8W7tJg", "full_address": "2149 W Wood Dr\nPhoenix, AZ 85029", "open": true, "categories": ["Sporting Goods", "Bikes", "Shopping"], "city": "Phoenix", "review_count": 5, "name": "Bike Doctor", "neighborhoods": [], "longitude": -112.10593299999999, "state": "AZ", "stars": 5.0, "latitude": 33.604053999999998, "type": "business"}
{"business_id": "3f_lyB6vFK48ukH6ScvLHg", "full_address": "1134 N Central Ave\nPhoenix, AZ 85004", "open": true, "categories": [], "city": "Phoenix", "review_count": 4, "name": "Valley Permaculture Alliance", "neighborhoods":

In [214]:
def raw(string):
    return "\"" + string + "\""

In [97]:
# from collections import Counter
# Counter(p).most_common(100)

In [215]:
food_categories = ["Restaurants", "Sandwiches", "Food", "Fries", "Burger", "Subs", "Coffee", "Pizza", "Grill", "Mexican", "Restaurant", "Restaurant", "Taco", "McDonalds"]
i = 0
food_businesses = []
potentials = []
while(i < len(json_str)-20):
    if json_str[i:i+13] == raw("business_id"):
        i += 15
        b_id = json_str[i:i+30].split()[0]
        categories = []
        while json_str[i:i+12] != raw("categories"):
            i += 1
        i += 14
        while json_str[i:i+6] != raw("city"):
            for s in food_categories:
                if json_str[i:i+len(s)+2] == raw(s):
                    food_businesses.append(b_id)
            i += 1
#         potential = json_str[i:i+120].split()
#         potential = [string.translate(None, "\"").translate(None, '\'').translate(None, ',') for string in potential]
#         potentials.append(potential)
#         print potential
#         if "Accountants" in potential:
#             print "found restaurant!!!\n\n\n"
    if i % 1000000 == 0:
        print i
    i += 1

0
1000000
4000000


In [216]:
fb = {i:1 for i in food_businesses}

In [234]:
# json_file = open('yelp_training_set/yelp_training_set_review.json')
# json_str = json_file.read()
# # json_data = json.loads(json_str)[0]
# print json_str[:10000]

In [295]:
reviews = []
for review in open('yelp_training_set/yelp_training_set_review.json', 'r'):
    reviews.append(json.loads(review))
    
food_categories = ["Restaurants", "Sandwiches", "Food", "Fries", "Burger", "Subs", "Coffee", "Pizza", "Grill", "Mexican", "Restaurant", "Restaurant", "Taco", "McDonalds"]
businesses = []
for business in open('yelp_training_set/yelp_training_set_business.json', 'r'):
    businesses.append(json.loads(business))
    
food_ids = {}
for business in businesses:
    if any(i in food_categories for i in business['categories']):
        food_ids[business['business_id']] = 1
        
review_texts = []
for review in reviews:
    if review['business_id'] in food_ids:
        review_texts.append(('{' + review['text'] + '}').lower().encode('ascii',errors='ignore').translate(None, 'ñ'))
        
path = 'yelp_100.txt'
text = open(path).read().lower()
chars = {i:1 for i in list(set(text))}

lengths = [100]#, 1000, 10000, len(review_texts)]
for l in xrange(1, 10):
    length = l * 100
    text_file = open("yelp_" + str(100) + "_" + str(l+2) + ".txt", "w")
    for review in xrange(100+length, 200+length):
        chars2 = {i:1 for i in list(set(review_texts[review]))}
        to_remove = []      
        for char in chars2.keys():
            if char not in chars:
                print(char)
                to_remove.append(char)
        cut_review = review_texts[review]
        for j in xrange(len(to_remove)):
            cut_review = cut_review.translate(None, to_remove[j])
        text_file.write(cut_review)

~
@
~
_
_
~
~
_
~
~
~
@
@


In [276]:
first_100 = ""
for review in xrange(100):
    first_100 += reviews[review]['text']

In [284]:
second_100 = ""
for review in xrange(100, 200):
    second_100 += reviews[review]['text']

In [285]:
chars1 = list(set(first_100))
chars2 = list(set(second_100))
# "ñ"

In [286]:
for i in chars2:
    if i not in chars1:
        print i
print len(chars1), len(chars2)

ñ
87 85


In [270]:
len(review_texts)

178744

In [267]:
review_texts[0]

'{my wife took me here on my birthday for breakfast and it was excellent.  the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  our waitress was excellent and our food arrived quickly on the semi-busy saturday morning.  it looked like the place fills up pretty quickly so the earlier you get here the better.\n\ndo yourself a favor and get their bloody mary.  it was phenomenal and simply the best i\'ve ever had.  i\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  it was amazing.\n\nwhile everything on the menu looks excellent, i had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  it came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  it was the best "toast" i\'ve ever had.\n\nanyway, i can\'t wait to go back!}'

In [218]:
fb.keys()[:10]

['"KwGQqUCpHHhOhVqyOL2QTA",',
 '"cBpJIOrVXotDI0XAZH_k0g",',
 '"lPtTXn7_VOwqypZs07j2nA",',
 '"zjBoVPJi27Ntm5neRuHGvg",',
 '"2I666dqzs1XB6xigIKa9bA",',
 '"6fR4ncWEfdMLk9AacDsRzQ",',
 '"seu8O6yPQ5Yz8kQv-KFnKQ",',
 '"Bv3g-9R5wUevZPP0jzihqg",',
 '"LjwXnAjusrwCZGsWV3Ue8Q",',
 '"qgVS6sBe7JNEI8otg0Y3dg",']

In [205]:
i = 0
reviews = []
ids = []
while(i < len(json_str)-30):
# while(i < 10000):
    if json_str[i:i+6] == '\"text\"':
        i += 8
        new = ""
        while(json_str[i:i+8] != ', \"type\"'):
            new += json_str[i]
            i += 1
            if i % 10000000 == 0:
                print i
#         potential = json_str[i:i+150].split()
#         for i, v in enumerate(potential):
#             print v,
#             if v == raw("business_id"):
#                 print potential[i+1]
        
#         potential = json_str[i:i+50].split()
#     print potential
        
#         while json_str[i:i+13] != raw("business_id"):
#             if i % 10000000 == 0:
#                 print i, len(reviews)
#             if json_str[i:i+7] == raw("votes"):
#                 break
#             i += 1
#         i += 14
#         b_id = json_str[i:i+30].split()[0]
#         if b_id in fb:
#             reviews.append(new)
        k = 0
        while json_str[i:i+13] != raw("business_id") and k < 100:
            i += 1
            k += 1
            if i % 10000000 == 0:
                print i, len(reviews)
            if k == 99:
                print "no business_id found"
        ids.append(json_str[i+15:i+14+25])
        reviews.append(new)        
    i += 1
    if i % 10000000 == 0:
        print i, len(reviews)

10000000 10597
20000000 21332
30000000
40000000 42653
50000000
60000000
70000000
80000000
90000000
110000000
120000000 127630
130000000
140000000 148680
150000000
160000000 170056
170000000 180735
180000000
190000000 202032
200000000
210000000


In [206]:
print len(fb.keys()), len(reviews)

5792 229907


In [174]:
len(reviews)

10

In [171]:
reviews

[([',',
   '"type":',
   '"review",',
   '"business_id":',
   '"9yKzy9PApeiPPOUJEtnvkg"}',
   '{"votes":',
   '{"funny":',
   '0,',
   '"useful":',
   '0,',
   '"cool":',
   '0},',
   '"user_id":',
   '"0a2KyEL0d3Yb1V6aivbIuQ",',
   '"rev'],
  '"My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\\n\\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\\n\\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their gr

In [73]:
text_file = open("yelp.txt", "w")
for review in reviews:
    text_file.write(review + "\n")

In [74]:
text_file.close()

In [75]:
n = open("nietzsche.txt", "r").read()

In [78]:
len(n)

600901

In [9]:
text_file = open("yelp_medium_demarcated.txt", "w")
for review in xrange(10000):
    text_file.write("{" + reviews[review] + "}")

In [10]:
text_file = open("yelp_small_demarcated.txt", "w")
for review in xrange(1000):
    text_file.write("{" + reviews[review] + "}")

In [10]:
text_file = open("yelp_tiny_demarcated.txt", "w")
for review in xrange(10):
    text_file.write("{" + reviews[review] + "}")

In [2]:
# Munging for character similarity

In [6]:
small = open("yelp_small.txt", "r").read()

In [12]:
small_ht = {}
for i in small:
    small_ht[i] = 1

In [14]:
len(list(small_ht))

90

In [15]:
medium_ht = {}
for i in open("yelp_medium.txt", "r").read():
    medium_ht[i] = 1

In [16]:
len(list(medium_ht))

94

In [17]:
for i in list(medium_ht):
    if i not in small_ht:
        print i

`
|
{
}
